In [1]:
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

train_data = pd.read_csv('chinese_mnist.csv')

Prepare for dataset

In [2]:
train_data

,suite_id,sample_id,code,value,character
0,1,1,10,9,九
1,1,10,10,9,九
2,1,2,10,9,九
3,1,3,10,9,九
4,1,4,10,9,九
...,...,...,...,...,...
14995,99,5,9,8,八
14996,99,6,9,8,八
14997,99,7,9,8,八
14998,99,8,9,8,八


In [3]:
# for i in range(15000):
#     if train_data["value"][i] > 10:
#         train_data = train_data.drop([i])
# train_data

In [4]:
def create_file_name(x):
    
    file_name = f"input_{x[0]}_{x[1]}_{x[2]}.jpg"
    return file_name

train_data["file_name"] = train_data.apply(create_file_name, axis=1)

In [5]:
numpy_data = np.zeros(shape=(15000,4096))
for i in range(15000):
    string = "data/" + train_data["file_name"][i]       # create string of path
    image = plt.imread(string).flatten()          # read the image and flattenone by one
    numpy_data[i] = image

In [6]:
new_train = pd.DataFrame(data=numpy_data, index=[i for i in range(1, 15001)], columns=[i for i in range(1, 4097)])
new_train

,1,2,3,4,5,6,7,8,9,10,...,4087,4088,4089,4090,4091,4092,4093,4094,4095,4096
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
X = new_train
y = train_data['value']

In [ ]:
# drop the rows with value > 10
for i in range(15000):
    if train_data["value"][i] > 10:
        X = X.drop([i])
X

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.2, random_state=42)

In [14]:
# Normalize input values
X_train = X_train/255.
X_test = X_test/255.

# # Generate 32x32 image set for use in LeNet
# cnn_x_train = np.pad(X_train, ((0,0),(2,2),(2,2))).reshape((-1, 32, 32, 1))
# cnn_x_test = np.pad(X_test, ((0,0),(2,2),(2,2))).reshape((-1, 32, 32, 1))

# Resize images for use in MLP
mlp_x_train = X_train.reshape((X_train.shape[0], 4096))
mlp_x_test = X_test.reshape((X_test.shape[0], 4096))

AttributeError: 'DataFrame' object has no attribute 'reshape'

In [9]:
X_train.shape

(12000, 4096)

In [19]:
# Implements the network using TensorFlow.Keras
def MLPNetwork(inputDim):
    # input layer (note that batches are already taken care of for you)
    x = keras.Input(shape=(inputDim,))
    
    # subsequent dense layers
    a1 = keras.layers.Dense(1024, activation='relu')(x)
    a2 = keras.layers.Dense(1024, activation='relu')(a1)
    
    # classification dense layer
    logits = keras.layers.Dense(15, activation='softmax')(a2)
    
    # Creates the model given the above structure
    model = keras.Model(inputs=x, outputs=logits)
    return model

MLPmod = MLPNetwork(4096)
MLPmod.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 4096)]            0         
_________________________________________________________________
dense_6 (Dense)              (None, 1024)              4195328   
_________________________________________________________________
dense_7 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_8 (Dense)              (None, 15)                15375     
Total params: 5,260,303
Trainable params: 5,260,303
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Sets up optimizer
opt = keras.optimizers.Adam(learning_rate=0.001)
MLPmod.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# The callback forces the network to train until it reaches 100% accuracy
hist = MLPmod.fit(X_train, y_train, epochs=20, shuffle=True)

Epoch 1/20


InvalidArgumentError:  Received a label value of 100000000 which is outside the valid range of [0, 15).  Label values: 2 100 100 8 7 5 4 4 1000 4 5 1000 1000 7 1000 7 10 1 8 3 3 100000000 3 5 6 1000 5 10000 6 8 10000 0
	 [[node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits (defined at <ipython-input-20-79207b249ed1>:6) ]] [Op:__inference_train_function_1622]

Function call stack:
train_function
